<a href="https://colab.research.google.com/github/Intruz01/COVID19_spread_prediction/blob/master/COVID19_spread_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## IMPORT LIBRARIES

import pandas as pd
import numpy as np
import datetime

In [0]:
## READ DATA FOR CASES, DEATH and RECOVERIES
# for this prediction I am using data posted and updated by CSSEGISandData on https://github.com/CSSEGISandData

df_confirmed_cases_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recoveries_raw = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')